<hr style="border:0.2px solid black"> </hr>

<figure>
  <IMG SRC="img/ntnu-norwegian-university-of-science-and-technology-vector-logo.png" WIDTH=250 ALIGN="right">
</figure>

**<ins>Course:</ins>** TVM4174 - Hydroinformatics for Smart Water Systems

# <ins>Group project:</ins> Working with SWMM in Python - Part 1: Analyze and modify the model
    
*Developed by Lucia Arce and Magdalena Jaurena*

<hr style="border:0.2px solid black"> </hr>


### What is SWMM?

https://www.epa.gov/water-research/storm-water-management-model-swmm

### Which packages exist to work with SWMM in Pyhton?

PySWMM: https://pyswmm.readthedocs.io/en/stable/overview.html


SWMMIO: https://swmmio.readthedocs.io/en/stable/index.html

#### In this notebook it is used SWMMIO to analyze and modify the model


In [ ]:
# SWMMIO installation

!pip install swmmio

In [ ]:
### Relevant packages for later use ### ###### REVISAR QUE USAMOS TODOS ########
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import swmmio
sns.set_style('whitegrid')

In [ ]:
### Import SWMM model ###

from swmmio import Model 
m = Model("SWMM_model.inp")


## 1- Analyze the structure of the network

In [ ]:
#### NODES ####

## Save junctions ##
nodes=m.inp.junctions
print (f'The information stored for the JUNCTIONS is: {nodes.columns}')
depth=pd.DataFrame([(nodes.idxmax()['MaxDepth'],nodes.max()['MaxDepth']), (nodes.idxmin()['MaxDepth'],nodes.min()['MaxDepth'])])
depth.columns=['Node','Value']
depth.index=['Max_depth','Min_depth']
depth.index.name='Type'
# print(nodes_coord)

## Save the coordinates of all the nodes in the model ##
nodes_coord=m.inp.coordinates
print (f'\nThe information stored for the NODES COORDINATES is: {nodes_coord.columns}')
# print(nodes_coord)

## Save outfalls ##
outfalls=m.inp.outfalls
print (f'\nThe information stored for the OUTFALLS is: {outfalls.columns}')
# print(outfalls)

## Save storages ##
storages=m.inp.storage
print (f'\nThe information stored for the STORAGES is: {storages.columns}')
# print(storages)

print(f"\nThere are {len(nodes)} nodes, {len(outfalls)} outfalls and {len(storages)} storage units in the network.")
print(f'\nThe maximum and minimum depth for the manholes (nodes) are: \n {depth}')


In [ ]:
#### LINKS ####

## Extract the data of every link in the model ##
links=m.links.dataframe       # Links include: conduits, weirs, orificies and pumps
print(f'The information stored for the links is: {links.columns}')
# links[['Shape','Geom1']]

## Extract only the conduits ##
conduits=m.inp.conduits
print (f'\nThe information stored for the conduits is: {conduits.columns}')
# print (conduits)

## Extract only the weirs ##
weirs=m.inp.weirs
print (f'\nThe information stored for the weirs is: {weirs.columns}')
#print (weirs)

## Extract only the orificies ##
orificies=m.inp.orifices
print (f'\nThe information stored for the orificies is: {orificies.columns}')
#print (orificies)

## Extract pumps ##
pumps=m.inp.pumps
print (f'\nThe information stored for the pumps is: {pumps.columns}')
#print (pumps)

In [ ]:
#### ORGANIZING THE INFORMATION OF THE MODEL - SUMMARY ####

total_length=links['Length'].sum()/1000
print(f'The total length of the pipes is {total_length:.2f} km.')
mean_length = links['Length'].mean()
print(f'\nThe average length of the pipes is {mean_length:.2f} m.')

print(f'\nThe model contains: {len(conduits):.0f} ' + '\033[1m'+'Pipes'+'\033[0m'+ f', {len(weirs):.0f} Weirs, {len(orificies):.0f} Orificies, {len(pumps):.0f} Pumps.')

## HISTOGRAM LENGTH OF PIPES ##

links['Length'].hist(bins=50, color='c')
plt.title('Histogram Length of pipes', fontsize=22)
plt.xlabel('Lenghts of pipe segments (m)', fontsize=16)
plt.ylabel('Number of pipes', fontsize=16)
plt.axvline(mean_length, color='k', linewidth=2, linestyle='--')
plt.text(mean_length-1,16, f'mean={mean_length:.2f}', horizontalalignment='right', fontsize=14)
plt.show()

## HISTOGRAM DIAMETER OF PIPES ##
mean_diam=links['Geom1'].mean()

links['Geom1'].hist(bins=10, color='c')
plt.title('Histogram Diameter of pipes', fontsize=22)
plt.xlabel('Diameter of pipe segments (m)', fontsize=16)
plt.ylabel('Number of pipes', fontsize=16)
plt.axvline(mean_diam, color='k', linewidth=2, linestyle='--')
plt.text(mean_diam-0.005,120, f'mean={mean_diam:.2f}', horizontalalignment='right', fontsize=14)
plt.show()


In [ ]:
### Curves of the system, this includes: control, diversion, pump, rating, shape, storage, tidal and weir curves ### 
curves=m.inp.curves
print (f'The information stored for the CURVES is: {curves.columns}')
# print(curves)

In [ ]:
links['Geom1']

In [ ]:
### SUBCATCHMENTS ###

subcatchments=m.inp.subcatchments
print(f'The information stored for the links is: {subcatchments.columns}')
# print (subcatchments['PercImperv'])

subareas=m.inp.subareas
print(f'\nThe information stored for the links is: {subareas.columns}')
# print(subareas)

##  Summary of surface characteristics for the whole study area ##
total_area=subcatchments.Area.sum()             
total_imperv=(subcatchments.Area * (subcatchments.PercImperv)/100).sum()
total_perv=total_area-total_imperv
imperv_perc=total_imperv/total_area *100

print(f"\nThere are {len(subcatchments)} subcatchments in the system.")
print(f"\nThe total area is: {total_area:0.2f} m^2.")
print(f'\nThe total impervious area is {total_imperv:0.2f} m^2 ({imperv_perc:0.2f}% of the total area), while the total pervious area is {total_perv:0.2f} m^2.')

## Plot of subcatchments impervious percentage ##
colors = sns.color_palette('viridis', len(subcatchments)) # Select colors to plot the different subcatchments
fig, ax = plt.subplots(figsize=(10, 6)) #figure size

subcatchments['PercImperv'].plot(kind= 'bar', color=colors, ax=ax)  # Bar plot

# Adjust plot presentation #

plt.title('Subcatchments impervious area percentaje', fontsize=22)
plt.xlabel('Subcatchment', fontsize=16)
plt.ylabel('Impervious %', fontsize=16)
plt.ylim((0, 100))
plt.show()

In [ ]:
### Code extracted from course "Urban Water Systems NTNU - 2021"  and adapted ###

polygons = m.inp.polygons
catchment_list = np.unique(polygons.index.to_numpy())  #Obtain catchment´s list
manhole_list = np.unique(nodes_coord.index.to_numpy()) #Obtain nodes´s list

fig, ax = plt.subplots(figsize = (10,10)) 

for catchment in catchment_list: 
    ax.fill(polygons.loc[catchment,"X"],polygons.loc[catchment,"Y"], ec = "burlywood", fc="floralwhite")
    
## Plot Conduits ##
for link in conduits.index: 
    inlet = nodes_coord.loc[conduits.loc[link, "InletNode"]] 
    outlet = nodes_coord.loc[conduits.loc[link, "OutletNode"]] 
    x_l = [inlet.X, outlet.X] 
    y_l = [inlet.Y, outlet.Y] 
    ax.plot(x_l,y_l, color = "k")

## Plot Weirs ##
for i in weirs.index: 
    inlet = nodes_coord.loc[weirs.loc[i, "InletNode"]] 
    outlet = nodes_coord.loc[weirs.loc[i, "OutletNode"]] 
    x_W = [inlet.X, outlet.X] 
    y_W = [inlet.Y, outlet.Y] 
    ax.plot(x_W,y_W, color = "b", linestyle='-.')

## Plot Nodes ##
for i in nodes.index: 
    coord_n = nodes_coord.loc[i] 
    ax.scatter(coord_n['X'], coord_n['Y'], color = "orangered", zorder=3 )
    
## Plot Outfalls ##
for i in outfalls.index: 
    coord_Out = nodes_coord.loc[i] 
    ax.scatter(coord_Out['X'],coord_Out['Y'], color = "k", marker='^', zorder=3)

## Plot Storages ##
for i in storages.index: 
    coord_S = nodes_coord.loc[i] 
    ax.scatter(coord_S['X'], coord_S['Y'], color = "red", marker='s', s=200, zorder=3)
    #plt.annotate(f'  Tank: {i}', (coord_S['X'], coord_S['Y']),) # Add tag to the node of the storage
    
### Notice that there are not orifices and pumps ploted, as the model does not contain any. 
### In case there were, the following lines should be included

## Plot Orificies ##
# for i in orificies.index:
#     inlet = nodes_coord.loc[orificies.loc[i, "InletNode"]] 
#     outlet = nodes_coord.loc[orificies.loc[i, "OutletNode"]] 
#     x_Orif = [inlet.X, outlet.X] 
#     y_Orif = [inlet.Y, outlet.Y] 
#     ax.plot(x_Orif,y_Orif, color = "k", linestyle='--')

## Plot Pumps ##
# for i in pumps.index:
#     inlet = nodes_coord.loc[orificies.loc[i, "InletNode"]] 
#     outlet = nodes_coord.loc[orificies.loc[i, "OutletNode"]] 
#     x_p = [inlet.X, outlet.X] 
#     y_p = [inlet.Y, outlet.Y] 
#     ax.plot(x_p,y_p, color = "c", linestyle=':')
  
    
plt.title('SWMM Model', fontsize=22)
plt.xlabel('x coordinate', fontsize=16)
plt.ylabel('y coorinate', fontsize=16)

# Create legend #
ax.plot(x_l,y_l, color = "k", label='Conduits')
ax.plot(x_W,y_W, color = "k", linestyle='-.', label='Weirs')
ax.scatter(coord_n['X'], coord_n['Y'], color = "orangered", label='Nodes', zorder=3 )
ax.scatter(coord_Out['X'],coord_Out['Y'], color = "k", marker='^', zorder=3, label='Outfalls')
ax.scatter(coord_S['X'], coord_S['Y'], color = "red", marker='s', s=200, label='Storages', zorder=3)

# Notice that there are not orifices and pumps ploted, as the model does not contain any. 
# In case there were, the following lines should be included
# ax.plot(x_Orif,y_Orif, color = "k", linestyle='--', label='Orificies')
# ax.plot(x_p,y_p, color = "c", linestyle=':', label='Pumps')

plt.legend(loc='best', frameon=False, title='Elements', title_fontsize=14)
plt.show()


In [ ]:
### Calculate slopes of the links ###

### Code extracted from course "Urban Water Systems NTNU - 2021"  and modified ###
slopes=pd.DataFrame([])
links_noslope=pd.DataFrame([])

for link in conduits.index:
    distance=conduits.loc[link, "Length"]
    try:
        inlet_z = nodes.loc[conduits.loc[link, "InletNode"], ["InvertElev"]] 
        outlet_z = nodes.loc[conduits.loc[link, "OutletNode"], ["InvertElev"]] 
        elevation = np.absolute(inlet_z.loc["InvertElev"]+conduits.loc[link,"InOffset"]-(outlet_z.loc["InvertElev"]+conduits.loc[link,"OutOffset"]))
        slope_perc = np.around( elevation / distance * 100, decimals = 2) 
        slopes=slopes.append([(link, slope_perc)])
        
# if slope_perc > slope_threshold: 
#             k+=1 
#             print("") 
#             print(link, f": the slope is higher than {slope_threshold}% a measure should be taken")
#             print(f"The slope is {slope_perc} % ") 
#             max_elevation = np.around(slope_threshold /100 * distance, decimals = 2) 
#             print(f"current elevation difference is {elevation}") 
#             print(f"it should be lower than {max_elevation}") 
#             print(f"increase inlet depth by {elevation-max_elevation}") 
            # if change is needed, make the change 
            #nodes.loc[conduits.loc[link, "InletNode"], "MaxDepth"] += elevation-max_elevation+0.1
        #max_slope=max()
   
    except KeyError: 
        print(link, " is linked to storage or outfall, manual operation is suggested")
        links_noslope=links_noslope.append([link])   

# if k==0: 
#     print("No problem with current slope") 
# else: 
#     print(f"We intented to solve a total of {k} slope problem, we should check again if the problem are solved")

slopes.columns=['Conduit','Slope%']
slopes.index=slopes['Conduit']
slopes.index.name='Conduit'
slopes.drop('Conduit', inplace=True, axis=1)

links_noslope.columns=['Conduit']
links_noslope.index=links_noslope['Conduit']
    

In [ ]:
### Code extracted from course "Urban Water Systems NTNU - 2021"  and adapted ###
### PLOT SLOPES ###

polygons = m.inp.polygons
catchment_list = np.unique(polygons.index.to_numpy())  #Obtain catchment´s list
manhole_list = np.unique(nodes_coord.index.to_numpy()) #Obtain nodes´s list

fig, ax = plt.subplots(figsize = (10,10)) 

for catchment in catchment_list: 
    ax.fill(polygons.loc[catchment,"X"],polygons.loc[catchment,"Y"],ec = "burlywood", fc="floralwhite")
    
## Plot Conduits Slope ##
for link in slopes.index: 
    inlet = nodes_coord.loc[conduits.loc[link, "InletNode"]] 
    outlet = nodes_coord.loc[conduits.loc[link, "OutletNode"]] 
    slope_x = [inlet.X, outlet.X] 
    slope_y = [inlet.Y, outlet.Y]
    if slopes.loc[link,'Slope%']>=20:
        ax.plot(slope_x, slope_y, c='r', linewidth=3)
        slope_x_20=slope_x
        slope_y_20=slope_y
    elif slopes.loc[link,'Slope%']<0.5:
        ax.plot(slope_x, slope_y, c='b', linewidth=3)
        slope_x_05=slope_x
        slope_y_05=slope_y
    else:
        ax.plot(slope_x, slope_y, c='gray', linewidth=3)
        slope_x_=slope_x
        slope_y_=slope_y

## Plot Conduits Linked to outfall or storage ##
for link in links_noslope.index: 
    inlet = nodes_coord.loc[conduits.loc[link, "InletNode"]] 
    outlet = nodes_coord.loc[conduits.loc[link, "OutletNode"]] 
    slope_x = [inlet.X, outlet.X] 
    slope_y = [inlet.Y, outlet.Y]
    ax.plot(slope_x, slope_y, c='k', linewidth=3) 
        
## Plot Weirs ##
for i in weirs.index: 
    inlet = nodes_coord.loc[weirs.loc[i, "InletNode"]] 
    outlet = nodes_coord.loc[weirs.loc[i, "OutletNode"]] 
    x_W = [inlet.X, outlet.X] 
    y_W = [inlet.Y, outlet.Y] 
    ax.plot(x_W,y_W, color = "b", linestyle='-.')

## Plot Nodes ##
for i in nodes.index: 
    coord_n = nodes_coord.loc[i] 
    ax.scatter(coord_n['X'], coord_n['Y'], color = "k", zorder=3 )
    
## Plot Outfalls ##
for i in outfalls.index: 
    coord_Out = nodes_coord.loc[i] 
    ax.scatter(coord_Out['X'],coord_Out['Y'], color = "k", marker='^', zorder=3)

## Plot Storages ##
for i in storages.index: 
    coord_S = nodes_coord.loc[i] 
    ax.scatter(coord_S['X'], coord_S['Y'], color = "red", marker='s', s=200, zorder=3)
    #plt.annotate(f'  Tank: {i}', (coord_S['X'], coord_S['Y']),) # Add tag to the node of the storage
    
### Notice that there are not orifices and pumps ploted, as the model does not contain any. 
### In case there were, the following lines should be included

## Plot Orificies ##
# for i in orificies.index:
#     inlet = nodes_coord.loc[orificies.loc[i, "InletNode"]] 
#     outlet = nodes_coord.loc[orificies.loc[i, "OutletNode"]] 
#     x_Orif = [inlet.X, outlet.X] 
#     y_Orif = [inlet.Y, outlet.Y] 
#     ax.plot(x_Orif,y_Orif, color = "k", linestyle='--')

## Plot Pumps ##
# for i in pumps.index:
#     inlet = nodes_coord.loc[orificies.loc[i, "InletNode"]] 
#     outlet = nodes_coord.loc[orificies.loc[i, "OutletNode"]] 
#     x_p = [inlet.X, outlet.X] 
#     y_p = [inlet.Y, outlet.Y] 
#     ax.plot(x_p,y_p, color = "c", linestyle=':')
  
    
plt.title('Slopes', fontsize=22)
plt.xlabel('x coordinate', fontsize=16)
plt.ylabel('y coorinate', fontsize=16)

# Create legend #
ax.plot(slope_x_20,slope_y_20, color = "r", linewidth=2 , label='Slope >= 20%')
ax.plot(slope_x_,slope_y_, color = "gray", linewidth=2 , label='0.5% < Slope <= 20%')
ax.plot(slope_x_05,slope_y_05, color = "b", linewidth=2 , label='Slope < 0.5%')
ax.plot(slope_x, slope_y, c='k', linewidth=2, label='No calculated slope')
ax.plot(x_W,y_W, color = "k", linestyle='-.', label='Weirs')
ax.scatter(coord_n['X'], coord_n['Y'], color = "k", label='Nodes', zorder=3 )
ax.scatter(coord_Out['X'],coord_Out['Y'], color = "k", marker='^', zorder=3, label='Outfalls')
ax.scatter(coord_S['X'], coord_S['Y'], color = "red", marker='s', s=200, label='Storages', zorder=3)

# Notice that there are not orifices and pumps ploted, as the model does not contain any. 
# In case there were, the following lines should be included
# ax.plot(x_Orif,y_Orif, color = "k", linestyle='--', label='Orificies')
# ax.plot(x_p,y_p, color = "c", linestyle=':', label='Pumps')

plt.legend(loc='best', frameon=False, title='Elements', title_fontsize=14)
plt.show()

## 2 - Hydrological module

In [ ]:
### See the options incorporated in the model to run the simulation ###

options=m.inp.options
options


In [ ]:
### See if there is any puntual inflow in the nodes ###

inflows = m.inp.inflows

In [ ]:
### See the raingages in the model ###

from swmmio.utils.dataframes import dataframe_from_rpt, get_link_coords, \
    create_dataframe_multi_index, get_inp_options_df, dataframe_from_inp

raingage = dataframe_from_inp('SWMM_model.inp', "RAINGAGES")
raingage

In [ ]:
### See the time series in the model and plot it ###

Dstorm=m.inp.timeseries
Dstorm["Value"] = pd.to_numeric(Dstorm["Value"], downcast="float")
print(Dstorm)

plt.figure(figsize=(15,6))
plt.plot(Dstorm['Time'], Dstorm['Value'], 'c', linewidth=5)

# Adjust plot presentation #
plt.title('Time series plot', fontsize=22)
plt.xlabel('Time (h)', fontsize=16)
plt.ylabel('Precipitation (mm)', fontsize=16)
plt.xlim(('0:00', '3:00'))
plt.ylim((0, 14))
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12)
plt.show()

## 3- Modify the network and settings, save the changes

This part aims to present some possible changes that can be done to the initial model. 

In [ ]:
## Modify Max_depth ##

node_modify='J10-004'                                                    # Node we want to modify
actual_value=nodes['MaxDepth'][node_modify]                              # Actual value of the variable
print(f'The actual max depth of node {node_modify} is {actual_value}')
nodes['MaxDepth'][node_modify]=3                                         # Specify Node, Property you want to change and value
nodes                                                                    # Print dataframe with all nodes

In [ ]:
## See if it was changed in the model ##
m.inp.junctions

In [ ]:
## Modify Link diameter ##

xsections=m.inp.xsections                                        # The conduit geometry is stored in this section in the model
conduit_modify='C10-004'                                         # Conduit we want to modify
actual_value_c=xsections['Geom1'][conduit_modify]                # Actual value of the variable
print(f'The actual diameter of the conduit {conduit_modify} is {actual_value_c}')
xsections['Geom1'][conduit_modify]=0.5                           # Specify Conduit, Property you want to change and value
xsections                                                        # Print dataframe with all nodes


In [ ]:
## See if it was changed in the model ##
m.inp.xsections

In [ ]:
### To change settings ###

# Change configuration of a defined value
print(options)
m.inp.options.loc['FLOW_UNITS', 'Value'] = 'CMS' 
m.inp.options.loc['FLOW_ROUTING', 'Value'] = 'KYNWAVE'
m.inp.options.loc['INFILTRATION', 'Value'] = 'GREEN_AMPT'   # Change configuration of a defined value


In [ ]:
## See if the OPTIONS were changed in the model ##
m.inp.options

In [ ]:
### Save model with modifications to a new version ###
m.inp.save("SWMM_model_NEW.inp")

In [ ]:
# from swmmio import Model 
# m_mod = Model("SWMM_model_NEW.inp")

In [ ]:
# m_mod.inp.junctions